In [2]:
import boto3

# Replace YOUR_ACCESS_KEY and YOUR_SECRET_KEY with your own access and secret keys
s3 = boto3.client('s3')

# Set the bucket name and the requester pays flag
bucket_name = 'biorxiv-src-monthly'
requester_pays = True

# Set the prefix to filter the objects by
prefix = ''

# Set the maximum number of objects to retrieve
max_objects = 10

# Call the list_objects_v2 function to retrieve the objects
objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, MaxKeys=max_objects, RequestPayer='requester')

# Iterate over the objects and print the object key for each object
for object in objects['Contents']:
    print(object['Key'])


Back_Content/
Back_Content/Batch_01/
Back_Content/Batch_01/0064b7c2-6c08-1014-9382-821844e0e0e5.meca
Back_Content/Batch_01/006789a2-6c19-1014-a2c5-9e42b34c170a.meca
Back_Content/Batch_01/00774734-6c28-1014-bf5f-9340390b63c3.meca
Back_Content/Batch_01/00ca4d31-6c03-1014-ba3a-a9ac76ede23b.meca
Back_Content/Batch_01/00ea436b-6c19-1014-b5f2-bd6047b47cb8.meca
Back_Content/Batch_01/00ebf884-6c08-1014-8d5a-cc84c59e05fe.meca
Back_Content/Batch_01/017a5006-6c19-1014-8fd1-acf2c586fd93.meca
Back_Content/Batch_01/017e3ddb-6c28-1014-be9e-bc3486017561.meca


In [3]:

# Initialize the pagination variables
paginator = s3.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix, RequestPayer='requester')


In [5]:
page = next(iter(page_iterator))

In [ ]:
page['Contents']

In [ ]:

# Iterate over the pages and print the object key for each object
for page in page_iterator:
    for object in page['Contents']:
        print(object['Key'])

In [14]:
import boto3
import zipfile
import xml.etree.ElementTree as ET
import os

class Bucket:
    def __init__(self, bucket_name:str, access_key=None, secret_key=None, requester_pays='True', prefix=''):
        # Create an S3 client using the given access and secret keys
        self.s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
        self.bucket_name = bucket_name
        self.requester_pays = requester_pays
        self.prefix = prefix

    
    def list_objects(self):
        # List the objects in the bucket, with the given prefix and requester pays flag
        paginator = self.s3_client.get_paginator('list_objects_v2')
        page_iterator = paginator.paginate(Bucket=self.bucket_name, Prefix=self.prefix, RequestPayer='requester')
        for page in page_iterator:
            for object in page['Contents']:
                yield object
    
    def download_object(self, object_key, filename):
        # Download the given object from the bucket and save it to the given filename
        self.s3_client.download_file(
            Bucket=self.bucket_name, 
            Key=object_key, 
            Filename=filename, 
            ExtraArgs={'RequestPayer': self.requester_pays})

    def parse_manifest(self, zip_filename):
        # Extract the zip file and parse the manifest.xml file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall()
        tree = ET.parse('manifest.xml')
        root = tree.getroot()
        
        # Return the manuscript title and the files
    def parse_manifest(self, zip_filename):
        # Extract the zip file and parse the manifest.xml file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall()
        tree = ET.parse('manifest.xml')
        root = tree.getroot()
        
        # Return the manuscript title and the files in the content folder
        title = root.find('./title')
        files = []
        for file in root.findall('./file'):
            name = file.find('name').text
            size = file.find('size').text
            files.append((name, size))
        return title.text, files
    
    def cleanup(self, zip_filename):
        # Clean up the extracted files
        zip_ref.close()
        os.remove(zip_filename)
        os.remove('manifest.xml')

# Example usage

# Replace YOUR_ACCESS_KEY and YOUR_SECRET_KEY with your own access and secret keys
access_key = 'YOUR_ACCESS_KEY'
secret_key = 'YOUR_SECRET_KEY'

# Set the bucket name and the requester pays flag
bucket_name = 'biorxiv-src-monthly'
requester_pays = True

# Set the prefix to filter the objects by
prefix = ''

# Create a Bucket instance
bucket = Bucket(bucket_name=bucket_name)

# Iterate over the objects in the bucket
for object in bucket.list_objects():
    # Download the object
    bucket.download_object(object['Key'], 'temp.zip')
    
    # Parse the manifest and print the title and file names and sizes
    title, files = bucket.parse_manifest('temp.zip')
    print(title)
    for name, size in files:
        print(f'{name}: {size} bytes')
    
    # Clean up
    bucket.cleanup('temp.zip')

    break



ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden

In [ ]:
ti